## Assignment 1: Reconstructing the path
In this assignment you will use Spark to compute the shortest path between two vertices. In the video, you have learned how to compute the distances between a source vertex and all other vertices in a graph. Now, your task is to reconstruct the shortest path, that is a sequence of vertices connected by the edges.

Dataset location: /data/twitter/twitter_sample.txt

Format: user_id \t follower_id

You can start with the code from the video:
```python
def parse_edge(s):
  user, follower = s.split("\t")
  return (int(user), int(follower))

def step(item):
  prev_v, prev_d, next_v = item[0], item[1][0], item[1][1]
  return (next_v, prev_d + 1)

def complete(item):
  v, old_d, new_d = item[0], item[1][0], item[1][1]
  return (v, old_d if old_d is not None else new_d)

n = 400  # number of partitions
edges = sc.textFile("/data/twitter/sample").map(parse_edge).cache()
forward_edges = edges.map(lambda e: (e[1], e[0])).partitionBy(n).persist()

x = 12
d = 0
distances = sc.parallelize([(x, d)]).partitionBy(n)
while True:
  candidates = distances.join(forward_edges, n).map(step)
  new_distances = distances.fullOuterJoin(candidates, n).map(complete, True).persist()
  count = new_distances.filter(lambda i: i[1] == d + 1).count()
  if count > 0:
    d += 1
    distances = new_distances
    print("d = ", d, "count = ", count)
  else:
    break
```
Your task is to find the shortest path between vertices 12 and 34. In case of multiple shortest paths (that is, disjoint paths with the same length), any will suffice. Output format is the sequence of vertices, delimited by a comma, without spaces. For example, the path “12 -> 42 -> 34” should be printed as
```
12,42,34
```
*Hint: before submitting, check your stopping criteria. In BFS, the search was exhaustive, and in this task your program may terminate earlier, thus saving some precious time.*

In [1]:
from pyspark import SparkConf, SparkContext
sc = SparkContext(conf=SparkConf().setAppName("MyApp").setMaster("local"))

In [2]:
def parse_edge(s):
    user, follower = s.split("\t")
    return (int(user), int(follower))

def step(item):
    prev_v, prev_d, next_v = item[0], item[1][0], item[1][1]
    return (next_v, prev_d + 1)

def complete(item):
    v, old_d, new_d = item[0], item[1][0], item[1][1]
    return (v, old_d if old_d is not None else new_d)

In [76]:
def path_step(item):
    last_vert, path, new_vert = item[0], item[1][0], item[1][1]
    last_vert = new_vert
    path += tuple([new_vert])
    return (last_vert, path)

In [96]:
n = 4  # number of partitions
edges = sc.textFile("/data/twitter/twitter_sample.txt").map(parse_edge).cache()
forward_edges = edges.map(lambda e: (e[1], e[0])).partitionBy(n).persist()

x_start = 12
x_end = 34

paths = sc.parallelize([ (x_start, tuple([x_start])) ]).partitionBy(n)

d = 0

while True:
    d += 1
    print 'Calculating distance = %d...' % d
    paths = paths.join(forward_edges, n).map(path_step)
    if paths.filter(lambda x: x[0] == x_end).count() > 0:
        break
        
paths = paths.filter(lambda x: x[0] == x_end).collect()

min_path = paths[0][1]
min_path_len = len(min_path)
for path in paths:
    path = path[1]
    if len(path) < min_path_len:
        min_path = path
        min_path_len = len(min_path)

print(','.join(map(str, min_path)))

Calculating distance = 1...
Calculating distance = 2...
Calculating distance = 3...
Calculating distance = 4...
Calculating distance = 5...
Calculating distance = 6...
Calculating distance = 7...
Calculating distance = 8...
12,422,53,52,107,20,23,274,34
